# Un zip for dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip "/content/drive/MyDrive/augmentation/dataSet.zip" -d "/content/dataset"

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
  inflating: /content/dataset/images/2013.png  
  inflating: /content/dataset/images/2014.png  
  inflating: /content/dataset/images/2015.png  
  inflating: /content/dataset/images/2016.png  
  inflating: /content/dataset/images/2017.png  
  inflating: /content/dataset/images/2018.png  
  inflating: /content/dataset/images/2019.png  
  inflating: /content/dataset/images/202.png  
  inflating: /content/dataset/images/2020.png  
  inflating: /content/dataset/images/2021.png  
  inflating: /content/dataset/images/2022.png  
  inflating: /content/dataset/images/2023.png  
  inflating: /content/dataset/images/2024.png  
  inflating: /content/dataset/images/2025.png  
  inflating: /content/dataset/images/2026.png  
  inflating: /content/dataset/images/2027.png  
  inflating: /content/dataset/images/2028.png  
  inflating: /content/dataset/images/2029.png  
  inflating: /content/dataset/images/203.png  
  inflating: /content/dataset/ima

# Data Augmentation

## Test function augmentation

In [ ]:
import albumentations as A
import numpy as np
import cv2

image = cv2.imread('/content/dog.jpg')
""" CenterCrop """
transform = A.Compose([A.CenterCrop(256, 256, p=1.0)])
augmented_image = transform(image=image, mask=image)
x = augmented_image["image"]
y = augmented_image["mask"]
cv2.imwrite("CenterCrop.png", x)
cv2.imwrite("CenterCrop.png", y)

""" CenterCrop """
transform = A.Compose([A.RandomRotate90(p=1.0)])
augmented_image = transform(image=image, mask=image)
x = augmented_image["image"]
y = augmented_image["mask"]
cv2.imwrite("RandomRotate90.png", x)
cv2.imwrite("RandomRotate90.png", y)

""" GridDistortion """
transform = A.Compose([A.GridDistortion(p=1.0)])
augmented_image = transform(image=image, mask=image)
x = augmented_image["image"]
y = augmented_image["mask"]
cv2.imwrite("GridDistortion.png", x)
cv2.imwrite("GridDistortion.png", y)

""" HorizontalFlip """
transform = A.Compose([A.HorizontalFlip(p=1.0)])
augmented_image = transform(image=image, mask=image)
x = augmented_image["image"]
y = augmented_image["mask"]
cv2.imwrite("HorizontalFlip.png", x)
cv2.imwrite("HorizontalFlip.png", y)

""" VerticalFlip """
transform = A.Compose([A.VerticalFlip(p=1.0)])
augmented_image = transform(image=image, mask=image)
x = augmented_image["image"]
y = augmented_image["mask"]
cv2.imwrite("VerticalFlip.png", x)
cv2.imwrite("VerticalFlip.png", y)

""" cv2 - VerticalFlip """
cv_VerticalFlip = cv2.flip(image, 1)
cv2.imwrite("cv2-VerticalFlip.png", x)

""" cv2 - HorizontalFlip """
cv_HorizontalFlip = cv2.flip(image, 0)
cv2.imwrite("cv2-HorizontalFlip.png", x)

""" cv2 - VerticalFlip - HorizontalFlip """
cv_HorizontalFlip = cv2.flip(image, -1)
cv2.imwrite("cv2-VerticalFlipHorizontalFlip.png", x)


True

## Augmentation

In [3]:
import os
import cv2
from tqdm import tqdm
from glob import glob
from albumentations import CenterCrop, RandomRotate90, GridDistortion, HorizontalFlip, VerticalFlip

def load_data(path):
    images = sorted(glob(os.path.join(path, "images/*")))
    masks = sorted(glob(os.path.join(path, "masks/*")))
    return images, masks

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def augment_data(images, masks, save_path, augment=True):
    H = 256
    W = 256

    for x, y in tqdm(zip(images, masks), total=len(images)):
        name = x.split("/")[-1].split(".")
        """ Extracting the name and extension of the image and the mask. """
        image_name = name[0]
        image_extn = name[1]

        name = y.split("/")[-1].split(".")
        mask_name = name[0]
        mask_extn = name[1]

        """ Reading image and mask. """
        x = cv2.imread(x, cv2.IMREAD_COLOR)
        y = cv2.imread(y, cv2.IMREAD_COLOR)

        """ Augmentation """
        if augment == True:
            aug = CenterCrop(H, W, p=1.0)
            augmented = aug(image=x, mask=y)
            x1 = augmented["image"]
            y1 = augmented["mask"]

            aug = RandomRotate90(p=1.0)
            augmented = aug(image=x, mask=y)
            x2 = augmented['image']
            y2 = augmented['mask']

            aug = GridDistortion(p=1.0)
            augmented = aug(image=x, mask=y)
            x3 = augmented['image']
            y3 = augmented['mask']

            aug = HorizontalFlip(p=1.0)
            augmented = aug(image=x, mask=y)
            x4 = augmented['image']
            y4 = augmented['mask']

            aug = VerticalFlip(p=1.0)
            augmented = aug(image=x, mask=y)
            x5 = augmented['image']
            y5 = augmented['mask']

            save_images = [x, x1, x2, x3, x4, x5]
            save_masks =  [y, y1, y2, y3, y4, y5]

        else:
            save_images = [x]
            save_masks = [y]

        """ Saving the image and mask. """
        idx = 0
        for i, m in zip(save_images, save_masks):
            i = cv2.resize(i, (W, H))
            m = cv2.resize(m, (W, H))

            tmp_img_name = f"{image_name}_{idx}.{image_extn}"
            tmp_mask_name = f"{mask_name}_{idx}.{mask_extn}"

            image_path = os.path.join(save_path, "images", tmp_img_name)
            mask_path = os.path.join(save_path, "masks", tmp_mask_name)

            cv2.imwrite(image_path, i)
            cv2.imwrite(mask_path, m)

            idx += 1

"""------------------------------------ Main ----------------------------------------- """
""" Loading original images and masks. """
path = "/content/dataset"
images, masks = load_data(path)
print(f"Số Hình Ban Đầu: {len(images)} - Số Mask Ban Đầu: {len(masks)}")

""" Creating folders. """
create_dir("new_data/images")
create_dir("new_data/masks")

""" Applying data augmentation. """
augment_data(images, masks, "new_data", augment=True)

""" Loading augmented images and masks. """
images, masks = load_data("new_data/")
print(f"Số Hình Đã Tăng Cường: {len(images)} - Số Mask Đã Tăng Cường : {len(masks)}")

Số Hình Ban Đầu: 3064 - Số Mask Ban Đầu: 3064


100%|██████████| 3064/3064 [02:11<00:00, 23.32it/s]


Số Hình Đã Tăng Cường: 18384 - Số Mask Đã Tăng Cường : 18384
